<a id='Q0'></a>
<center><a target="_blank" href="https://sit.academy/"><img src="https://drive.google.com/uc?id=1z0U84GYqhbWWpCenFajh8_8XFRGyOc3U" width="200" style="background:none; border:none; box-shadow:none;" /></a> </center>
<center> <h1> Data Science Toolkit : MIGROS challenge </h1> </center>
<p style="margin-bottom:1cm;"></p>
<center><h4>SIT Academy, 2022</h4></center>
<p style="margin-bottom:1cm;"></p>

<div style="background:#EEEDF5;border-top:0.1cm solid #EF475B;border-bottom:0.1cm solid #EF475B;">
    <div style="margin-left: 0.5cm;margin-top: 0.5cm;margin-bottom: 0.5cm;color:#303030">
        <p><strong>Aim of the Project:</strong> As a data scientist for Migros tasked with finding the best places to create new stores 
            1.Analyze market opportunities based on the density of existing stores
            2.Presence of competitors
            3.General population density to inform Migros’ development strategy.</p>
        
    
</nav>

In [ ]:
import json
import requests
import yaml
import pandas as pd
import time
import warnings
from pandas import json_normalize


# 1.Retrieve locations via Google Maps API:


In [ ]:
key_dict = yaml.load(open("credentials.yml"), Loader=yaml.FullLoader)
gmaps_key = key_dict["key"]
gmaps_key
api_key = gmaps_key

In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)
url="https://maps.googleapis.com/maps/api/place/textsearch/json?"
query="Migros, Bern"
df_all = pd.DataFrame()
stores = []
req = requests.get(url + "query=" + query + "&key=" + api_key)
places_json = req.json()
my_result = places_json["results"]
df_results = json_normalize(my_result)
df_all = df_all.append(df_results, ignore_index=True)
page_token = places_json['next_page_token']
#print(page_token)
#print(places_json.keys())
count=0
while True:
    req = requests.get(url + "query=" + query + "&key=" + api_key + "&pageToken=" + page_token)
    places_json = req.json()
    page_token = places_json['next_page_token'] if 'next_page_token' in places_json else None
    #print(page_token)
    my_result = places_json["results"]
    df_results = json_normalize(my_result)
    df_all = df_all.append(df_results, ignore_index=True)
    count += 1
    time.sleep(2)
    if count == 2:
        break

In [ ]:
df=df_all.drop(['icon', 'icon_background_color', 'icon_mask_base_uri','reference','geometry.viewport.northeast.lat','geometry.viewport.northeast.lng',
                'geometry.viewport.southwest.lat','business_status'],axis=1)
df_n=df.drop(['geometry.viewport.southwest.lng','plus_code.compound_code', 'plus_code.global_code','photos', 'types'],axis=1)
df_n.drop_duplicates()
df_all.to_csv('name.csv')

In [ ]:
df_geo=pd.read_csv('dataframe_pop_cleaned_2.csv')

In [ ]:
df_geo.head()
#df_geo.shape  (2172, 6)

In [ ]:
communes=df_geo['Name_of_commune']

In [ ]:
print(list(communes))

In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)
df_all = pd.DataFrame()
url="https://maps.googleapis.com/maps/api/place/textsearch/json?"
for commune in communes:
    query= "supermarkets, " + commune
    req = requests.get(url + "query=" + query + "&key=" + api_key)
    places_json = req.json()
    my_result = places_json["results"]
    df_results = json_normalize(my_result)
    df_all = df_all.append(df_results, ignore_index=True)
    if 'next_page_token' in places_json:
        page_token = places_json['next_page_token']
    #print(page_token)
    print(places_json.keys())
    count=0
    while 'next_page_token' in places_json:
        req = requests.get(url + "query=" + query + "&key=" + api_key + "&pageToken=" + page_token)
        places_json = req.json()
        page_token = places_json['next_page_token'] if 'next_page_token' in places_json else None
    #print(page_token)
        my_result = places_json["results"]
        df_results = json_normalize(my_result)
        df_all = df_all.append(df_results, ignore_index=True)
        count += 1
        time.sleep(2)
        if count == 3:
            break
    

In [ ]:
df_all.shape

In [ ]:
for i in df_all:
    print(i)

In [ ]:
df=df_all.drop(['icon', 'icon_background_color', 'icon_mask_base_uri','reference','geometry.viewport.northeast.lat','geometry.viewport.northeast.lng',
                'geometry.viewport.southwest.lat','business_status'],axis=1)
#df_all.drop(df_all.index, inplace=True)

In [ ]:
df_n=df.drop(['geometry.viewport.southwest.lng','plus_code.compound_code', 'plus_code.global_code','photos', 'types'],axis=1)


In [ ]:
df_n1=df_n.drop_duplicates().reset_index(drop=True)

In [ ]:
df_n1

In [ ]:
df_n['formatted_address'][1].split(',')

In [ ]:
df_sort=df_n1['formatted_address'].str.split(',')

In [ ]:
df_sort.head()


In [ ]:
df_n1.to_csv('/Users/rajendravrk/SIT/my-first-streamlitapp/Migros Challenge/data/processed/supermarket1.csv')